In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

2023-08-18 15:19:00.388929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load data
images = np.load("/Users/FranklinZhao/TensorFlowProjects/ImageBasedSneakerPrediction/data/cleaned/images.npy")
images = images.astype(np.float32)
images = images/255
prices = np.load("/Users/FranklinZhao/TensorFlowProjects/ImageBasedSneakerPrediction/data/cleaned/prices.npy")

data = tf.data.Dataset.from_tensor_slices((images, prices))

In [3]:
# split data
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [4]:
train = train.cache()
train = train.shuffle(train_size)
train = train.batch(80)
train = train.prefetch(80)

val = val.cache()
val = val.shuffle(val_size)
val = val.batch(80)
val = val.prefetch(80)


In [3]:
model = Sequential()

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape = (256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation = 'linear'))


In [6]:
model.compile('adam', tf.losses.MeanSquaredError())

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 64)        0

In [8]:
hist = model.fit(train, epochs=40, validation_data=val)

Epoch 1/40
10/10 [==============================] - 25s 2s/step - loss: 494064.8750 - accuracy: 0.0000e+00 - val_loss: 326681.3750 - val_accuracy: 0.0000e+00
Epoch 2/40
10/10 [==============================] - 17s 2s/step - loss: 297586.7812 - accuracy: 0.0000e+00 - val_loss: 283984.0938 - val_accuracy: 0.0000e+00
Epoch 3/40
10/10 [==============================] - 21s 2s/step - loss: 285224.4688 - accuracy: 0.0000e+00 - val_loss: 302803.0312 - val_accuracy: 0.0000e+00
Epoch 4/40
10/10 [==============================] - 23s 2s/step - loss: 285297.6562 - accuracy: 0.0000e+00 - val_loss: 261281.3281 - val_accuracy: 0.0000e+00
Epoch 5/40
10/10 [==============================] - 19s 2s/step - loss: 245801.3906 - accuracy: 0.0000e+00 - val_loss: 207143.7188 - val_accuracy: 0.0000e+00
Epoch 6/40
10/10 [==============================] - 20s 2s/step - loss: 195365.7344 - accuracy: 0.0000e+00 - val_loss: 180453.1094 - val_accuracy: 0.0000e+00
Epoch 7/40
10/10 [==============================] - 

In [288]:
model.save("/Users/FranklinZhao/TensorFlowProjects/ImageBasedSneakerPrediction/models/model_1.h5")

/Users/FranklinZhao/TensorFlowProjects/ImageBasedSneakerPrediction/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
